# Calibrate risk classes more

This notebook is for further calibration of the risk classes, based on continued assesment of current "production" daymet risk maps (circa January 2023).

We will first recompute the final risk dataset with three other moderate threshold / year propoertion combinations:

* 0.08 and 50% of years
* 0.12 for 33% of years
* 0.02 for 75% of years

(also decided to throw in a few more combos)

In [1]:
import shutil
import subprocess
from itertools import product
import numpy as np
import rasterio as rio
import xarray as xr
import rioxarray
import matplotlib.pyplot as plt
from wrf import PolarStereographic
from pyproj import Proj, Transformer
# project imports
from config import *
from classify_clip_mask import classify_risk

In [2]:
rm /atlas_scratch/kmredilla/beetles/moderate_risk_calibration_jan_2023/*

In [3]:
daymet_era = "1988-2017"

daymet_risk_fp = yearly_risk_dir.joinpath("yearly_risk_daymet_1980-2017.nc")
ds = xr.open_dataset(daymet_risk_fp).sel(year=slice(1988, 2017))

out_dir = scratch_dir.joinpath("moderate_risk_calibration_jan_2023")
out_dir.mkdir(exist_ok=True)

test_groups = [
    (0.06, 0.5),
    (0.08, 0.5),
    (0.12, 0.33),
    (0.1, 0.33),
    (0.08, 0.33),
    (0.02, 0.75),
    (0.04, 0.75),
]

for group in test_groups:
    med_thr, count_prop = group
    out_fp = out_dir.joinpath(f"risk_class_calibration_{med_thr}thr_{count_prop}prop_daymet.png")
    
    arr = ds["risk"].sel(snow="med").values
    out_arr = np.apply_along_axis(classify_risk, 0, arr, med_thr, count_prop)

    fig, ax = plt.subplots(1, 1, figsize=(18, 8))

    im = ax.imshow(out_arr, interpolation="none")
    plt.axis("off")
    ax.set_title(f"Beetle risk (Daymet)\nthreshold={med_thr}\ncount proportion={count_prop}", size=14, pad=-5)
    plt.savefig(out_fp, bbox_inches='tight', facecolor="white")
    plt.close()

Copy this to Google Drive for viewing:
```
rclone copy /atlas_scratch/kmredilla/beetles/moderate_risk_calibration_jan_2023/ google-drive:Beetles/moderate_risk_calibration_jan_2023
```